In [12]:
import sys
sys.path.insert(0, '..')

import asyncio
from pathlib import Path
import json

import importlib
if 'src.services.landingai_service' in sys.modules:
    importlib.reload(sys.modules['src.services.landingai_service'])

from src.services.landingai_service import LandingAIService

In [13]:
from dotenv import load_dotenv
import os

# Clear any existing environment variable first
if 'LANDING_AI_API_KEY' in os.environ:
    del os.environ['LANDING_AI_API_KEY']

env_path = Path("../.env")
print(f"Loading .env from: {env_path.absolute()}")
print(f"File exists: {env_path.exists()}")

# Force override with override=True
load_dotenv(env_path, override=True)

api_key = os.getenv("LANDING_AI_API_KEY")
print(f"API Key loaded: {api_key[:20] if api_key else 'NOT FOUND'}...")
print(f"Full key for verification: {api_key}")

service = LandingAIService()
print("Landing AI Service initialized successfully")
print(f"Confidence threshold: {service.confidence_threshold}")
print(f"Service API key: {service.api_key[:20]}...")

Loading .env from: c:\projects\HealthOps\backend\notebooks\..\.env
File exists: True
API Key loaded: pat_vTthl4om3xGRfrE1...
Full key for verification: pat_vTthl4om3xGRfrE1YY8BCH5Npx79zLh1
Landing AI Service initialized successfully
Confidence threshold: 0.5
Service API key: pat_vTthl4om3xGRfrE1...


In [14]:
pdf_path = "../data/sample_medical_record.pdf"

if Path(pdf_path).exists():
    print(f"✓ Found PDF: {pdf_path}")
    print(f"File size: {Path(pdf_path).stat().st_size / 1024:.2f} KB")
else:
    print(f"✗ PDF not found: {pdf_path}")
    print("Run the PDF generation cell first!")

✓ Found PDF: ../data/sample_medical_record.pdf
File size: 2.46 KB


In [15]:
result = await service.process_document(
    file_path=pdf_path,
    document_type="medical_record"
)

print(f"\nProcessing Status: {'SUCCESS' if result['success'] else 'FAILED'}")
print(f"Message: {result['message']}")
print(f"Processing Time: {result['processing_time']:.2f} seconds\n")


Processing Status: FAILED
Message: Failed to process document: API Error: 401 - {"errorCode":"UNAUTHORIZED","message":"Failed to find user record with apiKey: pat_vTthl4om3***","data":{}}
Processing Time: 0.93 seconds



In [ ]:
if result['success']:
    extracted = result['extracted_data']
    
    print("=" * 60)
    print("EXTRACTED TEXT")
    print("=" * 60)
    print(extracted.get('text', 'No text extracted')[:500])
    
    print("\n" + "=" * 60)
    print("KEY-VALUE PAIRS")
    print("=" * 60)
    kv_pairs = extracted.get('key_value_pairs', {})
    if kv_pairs:
        for key, value in list(kv_pairs.items())[:10]:
            print(f"{key}: {value}")
    else:
        print("No key-value pairs extracted")
    
    print("\n" + "=" * 60)
    print("TABLES")
    print("=" * 60)
    tables = extracted.get('tables', [])
    print(f"Found {len(tables)} table(s)")
    
    print("\n" + "=" * 60)
    print("METADATA")
    print("=" * 60)
    print(f"Confidence: {extracted.get('confidence', 0):.2%}")
    print(f"Document Type: {result.get('document_type', 'Unknown')}")
else:
    print(f"\n✗ Processing failed: {result['message']}")

In [ ]:
if result['success']:
    validation = await service.validate_extraction(result['extracted_data'])
    
    print("=" * 60)
    print("VALIDATION RESULTS")
    print("=" * 60)
    print(f"Status: {'PASSED' if validation['success'] else 'FAILED'}")
    print(f"Has Text: {validation['has_text']}")
    print(f"Has Tables: {validation['has_tables']}")
    print(f"Has Key-Value Pairs: {validation['has_key_value_pairs']}")
    print(f"Confidence Score: {validation['confidence_score']:.2%}")
    
    if validation.get('issues'):
        print("\nIssues Found:")
        for issue in validation['issues']:
            print(f"  - {issue}")
    else:
        print("\n✓ No issues found!")

In [ ]:
if result['success']:
    output_file = "../data/extraction_results.json"
    
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(result, f, indent=2, default=str)
    
    print(f"✓ Results saved to: {output_file}")

In [ ]:
print("\n" + "=" * 60)
print("TEST SUMMARY")
print("=" * 60)
print(f"Service Status: {'✓ Initialized' if service else '✗ Failed'}")
print(f"Processing Status: {'✓ Success' if result.get('success') else '✗ Failed'}")
print(f"Processing Time: {result.get('processing_time', 0):.2f}s")
print(f"Text Extracted: {'✓ Yes' if result.get('extracted_data', {}).get('text') else '✗ No'}")
print(f"Confidence: {result.get('extracted_data', {}).get('confidence', 0):.2%}")
print("=" * 60)